# マルチスレッド処理
通常、プログラムを実行すると上から順に実行されていきますが、マルチスレッド処理では複数の処理の流れを並行して進めることができます。

あるクラスのメソッド内で3秒待機したのちに標準出力にメッセージを出力させるプログラムがあったとします。<br>
それを通常のプログラムで実行させた場合と並列処理で実行させた場合での違いを見てみましょう。


### 以下は通常のプログラム

In [ ]:
class subClassA{
    public void run() {
        try{
            Thread.sleep(3000);
            System.out.println("サブクラスArunメソッド開始");
        }catch (InterruptedException e){
            e.printStackTrace();
        }
    
    }
}

class subClassB{
    public void run() {
        System.out.println("サブクラスBrunメソッド開始");
    }
}

In [ ]:
subClassA subA = new subClassA();
subClassB subB = new subClassB();
subA.run();
subB.run();

サブクラスArunメソッド開始<br>
サブクラスBrunメソッド開始

という風に出力されていますが、当り前ですよね。
では並列処理ではどうなるでしょうか。
### 並列処理を実装したプログラム


In [ ]:
class subA extends Thread {
    public void run() {
       try{
           sleep(3000);
           System.out.println("サブスレッドA開始");         
       }catch(InterruptedException e){
           e.printStackTrace();
       } 
    }
}

class subB extends Thread {
    public void run() {
        System.out.println("サブスレッドB開始");
    }
}

In [ ]:
subA threadA = new subA();
subB threadB = new subB();
threadA.start();
threadB.start();

threadA.join();
threadB.join();

サブスレッドB開始<br>
サブスレッドA開始

AとBの出力が逆になりました。<br>
subAの```sleep(3000)```(3秒待機)している間に<br>
subBの```System.out.println("サブスレッドB開始");```が実行されています。

このように、複数の処理を同時に実行できるのがマルチスレッドです。<br>

では、先程のマルチスレッドのソースコードの中身を詳しくみていきましょう。

```java
class subA extends Thread {
    public void run() {
       try{
           sleep(3000);
           System.out.println("サブスレッドA開始");         
       }catch(InterruptedException e){
           e.printStackTrace();
       } 
    }
}
class subB extends Thread {
    public void run() {
        System.out.println("サブスレッドB開始");
    }
}
```

subAクラスとsubBクラスはThreadというクラスを継承しており、Threadクラスは並列処理を行うためのクラスです。<br>
subA,subBクラスはThreadクラスのrun()メソッドをそれぞれオーバーライドして実装しています。

次に実行しているところを見ていきます。

```java
subA threadA = new subA();
subB threadB = new subB();
threadA.start();
threadB.start();

threadA.join();
threadB.join();
```

まずはsubA,subBクラスをインスタンス化しています。<br>
次に各サブクラスのstart()メソッドを実行しています。<br>
Threadクラスではstart()メソッドを実行するとサブスレッドでrun()メソッドの処理が始まります。<br>

最後のjoinメソッドでは各スレッドが終了するのを待っています。
今回はjupyter notebook上の都合でjoinメソッドを呼ぶ必要がありますが、実行環境やプログラムによっては呼ぶ必要がない場合もあります。

## Runnableインターフェースを使用する方法
ここまではThreadクラスを継承してマルチスレッドを利用する方法を記載しました。  
Javaではもう一つRunnableインターフェースを使用する方法があります。
<br>
スレッドで動かしたい処理をjava.lang.Runnableの実装クラスとして作り、<br>そのRunnableをThreadクラスのインスタンスに動かしてもらうことです。<br>

Runnableとは「実行(run)できる(able)もの」という意味のインターフェイスで、抽象メソッドは戻り値void、引数なしのrunだけです。<br>Runnableはスレッドに限らず、Javaで何か動かしたい処理を表現・実装するのにもよく使われます。
```java
class RunnableTest implements Runnable {
    @Override
    public void run() {
        for (int i = 0; i < 3; i++) {
            System.out.println("Runnable count: " + i);
        }
    }
}
```
Runnableインターフェースはrun()メソッドのみを持ち、run()メソッドをオーバライドして処理を記載して利用します。 <br>  
呼び出し側は、ThreadクラスのコンストラクタThread(Runnable target)を使います。<br>引数にはRunnableインターフェースを実装したクラスを指定します。<br>後はThreadクラスのstartメソッドをインスタンスに対して呼び出すことでスレッドが開始されます。
```java
public class ThreadExecutor {
 {
    public static void main(String[] args) {
        Thread thread = new Thread(new RunnableTest());
        thread.start();
        System.out.println("ThreadExecutor!");
    }
}
```
ThreadExecutorの実行結果は
```
ThreadExecutor!
Runnable count: 0
Runnable count: 1
Runnable count: 2
```
となります。
`ThreadExecutor!`が先に出力されており、RunnableTestのrun()メソッドが別スレッドで動いているのが分かります。


# 問題
マルチスレッド動作するプログラムを書いてみましょう